<a href="https://colab.research.google.com/github/IldarAltynbaev/MLSnippetSimpleNeuroNwtwork/blob/TestingColab/StepikRanForContest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
import pandas as pd
import numpy as np
from google.colab import drive
from datetime import timedelta

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [110]:
def count_correct(series):
    return (series == 'correct').sum()
def count_wrong(series):
    return (series == 'wrong').sum()
def count_viewed(series):
    return (series == 'viewed').sum()
def count_passed(series):
    return (series == 'passed').sum()
def count_discovered(series):
    return (series == 'discovered').sum()
def count_started_attempt(series):
    return (series == 'started_attempt').sum()


In [111]:
events_dataFrame = pd.read_csv('/content/drive/MyDrive/DS data/event_data_train.csv')
events_dataFrame['date_time'] = pd.to_datetime(events_dataFrame['timestamp'],unit='s')
events_dataFrame.sort_values('date_time');
#events_dataFrame = events_dataFrame.rename(columns={'user_id':'user_action'})
events_dataFrame

,step_id,timestamp,action,user_id,date_time
0,32815,1434340848,viewed,17632,2015-06-15 04:00:48
1,32815,1434340848,passed,17632,2015-06-15 04:00:48
2,32815,1434340848,discovered,17632,2015-06-15 04:00:48
3,32811,1434340895,discovered,17632,2015-06-15 04:01:35
4,32811,1434340895,viewed,17632,2015-06-15 04:01:35
...,...,...,...,...,...
3480698,33375,1526772685,viewed,11677,2018-05-19 23:31:25
3480699,32815,1526772750,viewed,5198,2018-05-19 23:32:30
3480700,32815,1526772750,discovered,5198,2018-05-19 23:32:30
3480701,32815,1526772750,passed,5198,2018-05-19 23:32:30


In [112]:
events_dataFrame[(events_dataFrame.user_id == 3) & (events_dataFrame.action == 'viewed')]

,step_id,timestamp,action,user_id,date_time
7363,32815,1434358476,viewed,3,2015-06-15 08:54:36
7367,32811,1434358481,viewed,3,2015-06-15 08:54:41
7390,32929,1434358504,viewed,3,2015-06-15 08:55:04
7391,32811,1434358511,viewed,3,2015-06-15 08:55:11
7392,32815,1434358512,viewed,3,2015-06-15 08:55:12
...,...,...,...,...,...
741523,33124,1444581517,viewed,3,2015-10-11 16:38:37
741525,33127,1444581576,viewed,3,2015-10-11 16:39:36
741529,33334,1444581578,viewed,3,2015-10-11 16:39:38
741532,33126,1444581586,viewed,3,2015-10-11 16:39:46


In [113]:
submission_df = pd.read_csv('/content/drive/MyDrive/DS data/submissions_data_train.csv')
submission_df['date_time'] = pd.to_datetime(events_dataFrame['timestamp'],unit='s')
submission_df.sort_values('date_time');

In [114]:
submission_df[submission_df.user_id == 5198]

,step_id,timestamp,submission_status,user_id,date_time


In [115]:
submission_count_correct = submission_df.groupby(["user_id"])['submission_status'].apply(count_correct).reset_index(name='count_correct')
submission_count_wrong = submission_df.groupby(['user_id'])['submission_status'].apply(count_wrong).reset_index(name='count_wrong')

In [116]:
user_data = pd.merge(events_dataFrame, submission_count_correct, how='left', on='user_id')
user_data = pd.merge(user_data, submission_count_wrong, how='left', on='user_id')

In [117]:
action_viewed = user_data.groupby(['user_id'])['action'].apply(count_viewed).reset_index(name='count_viewed')
action_passed = user_data.groupby(['user_id'])['action'].apply(count_passed).reset_index(name='count_passed')
action_discovered = user_data.groupby(['user_id'])['action'].apply(count_discovered).reset_index(name='count_discovered')
action_started_attempt = user_data.groupby(['user_id'])['action'].apply(count_started_attempt).reset_index(name='count_started_attempt')

In [118]:
user_data = pd.merge(user_data, action_viewed, how='left', on='user_id')
user_data = pd.merge(user_data, action_passed, how='left', on='user_id')
user_data = pd.merge(user_data, action_discovered, how='left', on='user_id')
user_data = pd.merge(user_data, action_started_attempt, how='left', on='user_id')

In [119]:
user_data['course_passed'] = user_data.count_correct > 40

In [120]:
user_data = user_data.fillna(0)

In [121]:
user_data

,step_id,timestamp,action,user_id,date_time,count_correct,count_wrong,count_viewed,count_passed,count_discovered,count_started_attempt,course_passed
0,32815,1434340848,viewed,17632,2015-06-15 04:00:48,31.0,278.0,555,86,119,149,False
1,32815,1434340848,passed,17632,2015-06-15 04:00:48,31.0,278.0,555,86,119,149,False
2,32815,1434340848,discovered,17632,2015-06-15 04:00:48,31.0,278.0,555,86,119,149,False
3,32811,1434340895,discovered,17632,2015-06-15 04:01:35,31.0,278.0,555,86,119,149,False
4,32811,1434340895,viewed,17632,2015-06-15 04:01:35,31.0,278.0,555,86,119,149,False
...,...,...,...,...,...,...,...,...,...,...,...,...
3480698,33375,1526772685,viewed,11677,2018-05-19 23:31:25,30.0,20.0,377,80,90,39,False
3480699,32815,1526772750,viewed,5198,2018-05-19 23:32:30,0.0,0.0,1,1,1,0,False
3480700,32815,1526772750,discovered,5198,2018-05-19 23:32:30,0.0,0.0,1,1,1,0,False
3480701,32815,1526772750,passed,5198,2018-05-19 23:32:30,0.0,0.0,1,1,1,0,False
